In [2]:
import numpy as np
import math

T_cano = np.eye(4)

T0 = np.array([[0.9999500513076782, 0.004810694605112076, 0.008763475343585014, 0.010515931993722916],
        [0.008762343786656857, 0.0002564578899182379, -0.9999614953994751, -1.1999539136886597],
        [-0.004812757018953562, 0.9999883770942688, 0.00021429210028145462, 0.00025705056032165885],
        [0.0, 0.0, 0.0, 1.0]])

T1 = np.array([ [0.861971378326416, -0.24816764891147614, 0.442061185836792, 0.5305169224739075],
        [0.5069567561149597, 0.42257794737815857, -0.751280665397644, -0.9016107320785522],
        [-0.0003617570619098842, 0.8716884255409241, 0.49006035923957825, 0.5878257155418396],
        [0.0, 0.0, 0.0, 1.0]])

T2 = np.array([[-0.008050278760492802, 0.34757664799690247, 0.9376169443130493, 1.1250077486038208],
        [0.9999675750732422, 0.0030544146429747343, 0.007453335449099541, 0.008942931890487671],
        [-0.00027326561394147575, 0.9376466274261475, -0.3475899398326874, -0.41733309626579285],
        [0.0, 0.0, 0.0, 1.0]])

T3 = np.array([[-0.8660249710083008, -0.2501213252544403, 0.4329434335231781, 0.5195764899253845],
        [0.4999733865261078, -0.42414551973342896, 0.7550675868988037, 0.9061554074287415],
        [-0.005227487534284592, 0.870367705821991, 0.49237459897994995, 0.5906028747558594],
        [0.0, 0.0, 0.0, 1.0]])

T4 = np.array([[-0.8700066804885864, -0.1741916537284851, -0.46124354004859924, -0.5534254312515259],
        [-0.4930136799812317, 0.2977108061313629, 0.8174996376037598, 0.9808858036994934],
        [-0.0050844247452914715, 0.9386296272277832, -0.34488925337791443, -0.4140926003456116],
        [0.0, 0.0, 0.0, 1.0]])

T5 = np.array([[-0.008049866184592247, 0.4948376417160034, -0.8689481019973755, -1.0428248643875122],
        [-0.9999675750732422, -0.003890044754371047, 0.007048365660011768, 0.008458686992526054],
        [0.00010754966933745891, 0.8689767122268677, 0.4948529005050659, 0.5935773849487305],
        [0.0, 0.0, 0.0, 1.0]])

T6 = np.array([[0.866024911403656, -0.17087915539741516, -0.4698948264122009, -0.5638080835342407],
        [-0.49997571110725403, -0.3053785562515259, -0.8104123473167419, -0.9723791480064392],
        [-0.005013220012187958, 0.9367733001708984, -0.3499008119106293, -0.4201057553291321],
        [0.0, 0.0, 0.0, 1.0]])

In [3]:
import bpy
import os
import numpy as np

# Empty the scene
bpy.ops.object.select_all(action='SELECT')
bpy.ops.object.delete(use_global=False)

# setup
scene = bpy.data.scenes[0]
scene.use_nodes = True
scene.render.resolution_x = 512
scene.render.resolution_y = 512
scene.render.resolution_percentage = 100
bpy.context.scene.world.use_nodes = False
bpy.context.scene.render.engine = 'CYCLES'
# bpy.context.scene.render.image_settings.color_mode = 'RGBA'
# bpy.context.scene.render.image_settings.file_format = 'JPEG'
bpy.context.scene.render.image_settings.quality = 100

# Build scene
bpy.context.scene.world.color = (0, 0, 0)
bpy.context.scene.render.film_transparent = True
# self.scene = Scene()

# set up light
light_data = bpy.data.lights.new(name="my-light-data", type='POINT')
light_data.energy = 100
light_object = bpy.data.objects.new(name="my-light", object_data=light_data)
light_object.location = (0, 0, 3)
bpy.context.collection.objects.link(light_object)

# set up camera
# Create a new camera data
base_path = '/mnt/qb/work/ponsmoll/yxue80/project/RVH-123/rvh123/rendering_top/125611487366942/0'
cam_pose_all = []
for i in range(7):
    var_name = 'T' + str(i)
    cam_pose = locals()[var_name] if var_name in locals() else None
    if cam_pose is not None:
        print(f'{var_name} = {cam_pose}')
    R_, T_ = cam_pose[:3, :3], cam_pose[:3, -1]

    T = -R_.T @ T_

    cam_pose_new = np.zeros((4, 4))
    cam_pose_new[:3, :3] = R_
    cam_pose_new[:3, -1] = T

    cam_data = bpy.data.cameras.new('Camera')
    cam_obj = bpy.data.objects.new('Camera', cam_data)
    cam_obj.matrix_world = cam_pose_new
    cam_obj.location = T * 5

    print(cam_obj.rotation_euler)
    print(T)

    bpy.context.collection.objects.link(cam_obj)
    # Set the active camera
    bpy.context.scene.camera = cam_obj
    # bpy.context.scene.view_layers['View Layer'].use_pass_combined = True
    # bpy.context.scene.view_layers['View Layer'].use_pass_diffuse_color = True
    # bpy.context.scene.view_layers['View Layer'].use_pass_z = True


bpy.context.scene.use_nodes = True
scene_node_tree = bpy.context.scene.node_tree
for n in scene_node_tree.nodes:
    scene_node_tree.nodes.remove(n)

render_layer = scene_node_tree.nodes.new(type="CompositorNodeRLayers")
output_rgb = scene_node_tree.nodes.new(type="CompositorNodeOutputFile")
output_rgb.base_path = ''
output_rgb.file_slots[0].use_node_format = True
output_rgb.format.file_format = "JPEG" 
output_rgb.format.color_mode = 'RGB'
scene_node_tree.links.new(render_layer.outputs['Image'], output_rgb.inputs['Image'])

output_depth = scene_node_tree.nodes.new(type="CompositorNodeOutputFile")
output_depth.base_path = ''
output_depth.file_slots[0].use_node_format = True
output_depth.format.file_format = "OPEN_EXR"
output_depth.format.color_depth = '16'
scene_node_tree.links.new(render_layer.outputs['Depth'], output_depth.inputs['Image'])

# rendering setting
bpy.context.scene.cycles.samples = 128
bpy.context.scene.cycles.device = 'GPU'
# Enable CUDA
bpy.context.preferences.addons['cycles'].preferences.compute_device_type = 'CUDA'

bpy.ops.object.select_all(action='DESELECT')


bpy.ops.wm.save_as_mainfile(filepath='/home/yuxuan/project/zero123plus/cams.blend')

T0 = [[ 9.99950051e-01  4.81069461e-03  8.76347534e-03  1.05159320e-02]
 [ 8.76234379e-03  2.56457890e-04 -9.99961495e-01 -1.19995391e+00]
 [-4.81275702e-03  9.99988377e-01  2.14292100e-04  2.57050560e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
<Euler (x=-1.5706, y=-0.0088, z=0.0048), order='XYZ'>
[ 2.39105102e-07  1.01138747e-07 -1.19999992e+00]
T1 = [[ 8.61971378e-01 -2.48167649e-01  4.42061186e-01  5.30516922e-01]
 [ 5.06956756e-01  4.22577947e-01 -7.51280665e-01 -9.01610732e-01]
 [-3.61757062e-04  8.71688426e-01  4.90060359e-01  5.87825716e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
<Euler (x=-0.9928, y=-0.4579, z=-0.2803), order='XYZ'>
[-1.00773576e-07  2.57077381e-04 -1.19995373e+00]
T2 = [[-8.05027876e-03  3.47576648e-01  9.37616944e-01  1.12500775e+00]
 [ 9.99967575e-01  3.05441464e-03  7.45333545e-03  8.94293189e-03]
 [-2.73265614e-04  9.37646627e-01 -3.47589940e-01 -4.17333096e-01]
 [ 0.00000000e+00  0.00000000e+00  0.

{'FINISHED'}

In [12]:
# test dataloader
import torch
import os
import numpy as np
from typing import Tuple


def cartesian_to_spherical(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    z = np.sqrt(xy + xyz[:,2]**2)
    theta = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    azimuth = np.arctan2(xyz[:,1], xyz[:,0])
    return np.array([theta, azimuth, z])

cam_pose_all = []
for i in range(7):
    var_name = 'T' + str(i)
    cam_pose = locals()[var_name] if var_name in locals() else None
    R_, T_ = cam_pose[:3, :3], cam_pose[:3, -1] * 5

    cam_pose_old = np.eye(4)
    cam_pose_old[:3, :3] = R_
    cam_pose_old[:3, -1] = T_

    swap_y_z = np.array([[1, 0, 0, 0],
                        [0, 0, -1, 0],
                        [0, -1, 0, 0],
                        [0, 0, 0, 1]])
    deg180 = np.deg2rad(180)
    rot_z = np.array([[np.cos(deg180), -np.sin(deg180), 0],
            [np.sin(deg180), np.cos(deg180), 0],
            [0, 0, 1]])
    
    c2w = swap_y_z @ cam_pose_old

    t = c2w[:3,-1]  # Extract translation of the camera
    r = c2w[:3, :3] @ rot_z # Extract rotation matrix of the camera

    t = t @ r # Make rotation local

    cam_pose_new = np.eye(4)
    cam_pose_new[:3, :3] = r
    cam_pose_new[:3, -1] = t

    cam_pose_all.append(torch.from_numpy(cam_pose_new).float())

cam_pose_all = torch.stack(cam_pose_all)

from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.renderer.cameras import (
    FoVPerspectiveCameras,
)
import torch

# set for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

R_abs = cam_pose_all[:, :3, :3].float()
T_abs = cam_pose_all[:, :3, 3].float() 

cameras_absolute_gt = FoVPerspectiveCameras(
    R = R_abs.to(device),
    T = T_abs.to(device),
    device = device,
)

plot = plot_scene({"k": {i: camera for i, camera in enumerate(cameras_absolute_gt)}}, camera_scale=1)
plot.layout.scene.aspectmode = "data"
plot.show()